# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [15]:
# Step 1: Install pyowm if not already installed
!pip install pyowm


In [16]:
# Step 2: Import the OWM library and authenticate with your real API key
from pyowm.owm import OWM

owm = OWM('49468e95f98bef66f2a726727d5b8e15')  # ✅ Your real OpenWeatherMap key
mgr = owm.weather_manager()

# Step 3: Get 3-hour forecast for a specific city
forecast = mgr.forecast_at_place("London", '3h')

# Step 4: Print the first 3 forecast entries (time and temperature)
for entry in forecast.forecast.weathers[:3]:
    temp = entry.temperature('celsius')['temp']
    time = entry.reference_time('iso')
    print(f"{time} → {temp}°C")



2025-05-15 15:00:00+00:00 → 13.17°C
2025-05-15 18:00:00+00:00 → 13.78°C
2025-05-15 21:00:00+00:00 → 12.18°C


## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [18]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 286.14,
  'feels_like': 285.35,
  'temp_min': 285.68,
  'temp_max': 287.04,
  'pressure': 1024,
  'humidity': 71,
  'sea_level': 1024,
  'grnd_level': 1020},
 'visibility': 10000,
 'wind': {'speed': 2.68, 'deg': 51, 'gust': 5.36},
 'clouds': {'all': 81},
 'dt': 1747318411,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1747282115,
  'sunset': 1747338318},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [19]:
#your code here

from pyowm.owm import OWM

# Initialize PyOWM with your actual OpenWeatherMap API key
owm = OWM('49468e95f98bef66f2a726727d5b8e15')

# Create a weather manager instance
mgr = owm.weather_manager()



## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [20]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 286.32°C, Humidity: 70%, Wind Speed: 2.24 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [21]:
#your code here

# Get current weather for London using pyowm
observation = mgr.weather_at_place("London")
weather = observation.weather

# Extract values
temperature = weather.temperature('celsius')['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

# Display output
print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")


Temperature: 13.17°C, Humidity: 70%, Wind Speed: 2.24 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [23]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [24]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-05-15 15:00:00, Temp: 290.95°C
Time: 2025-05-15 18:00:00, Temp: 292.11°C
Time: 2025-05-15 21:00:00, Temp: 294.06°C
Time: 2025-05-16 00:00:00, Temp: 292.22°C
Time: 2025-05-16 03:00:00, Temp: 290.82°C
Time: 2025-05-16 06:00:00, Temp: 289.71°C
Time: 2025-05-16 09:00:00, Temp: 289.93°C
Time: 2025-05-16 12:00:00, Temp: 291.93°C
Time: 2025-05-16 15:00:00, Temp: 295.76°C
Time: 2025-05-16 18:00:00, Temp: 296.21°C
Time: 2025-05-16 21:00:00, Temp: 294.45°C
Time: 2025-05-17 00:00:00, Temp: 292.13°C
Time: 2025-05-17 03:00:00, Temp: 291.35°C
Time: 2025-05-17 06:00:00, Temp: 291.58°C
Time: 2025-05-17 09:00:00, Temp: 291.27°C
Time: 2025-05-17 12:00:00, Temp: 292.75°C
Time: 2025-05-17 15:00:00, Temp: 296.7°C
Time: 2025-05-17 18:00:00, Temp: 299.11°C
Time: 2025-05-17 21:00:00, Temp: 299.96°C
Time: 2025-05-18 00:00:00, Temp: 297.98°C
Time: 2025-05-18 03:00:00, Temp: 295.98°C
Time: 2025-05-18 06:00:00, Temp: 292.06°C
Time: 2025-05-18 09:00:00, Temp: 289.47°C
Time: 2025-05-18 12:00:00, Temp: 29

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [25]:
#your code here

# Get 3-hour forecast directly for New York (no need to geocode manually)
forecast = mgr.forecast_at_place("New York", "3h")

# Print the first 3 forecasted entries: time and temperature
for entry in forecast.forecast.weathers[:3]:
    time = entry.reference_time('iso')
    temp = entry.temperature('celsius')['temp']
    print(f"{time} → {temp}°C")


2025-05-15 15:00:00+00:00 → 17.8°C
2025-05-15 18:00:00+00:00 → 18.96°C
2025-05-15 21:00:00+00:00 → 20.91°C


## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [27]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 286.32°C, Fahrenheit: 547.376°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [28]:
#your code here

# Get current weather observation for London
observation = mgr.weather_at_place("London")
weather = observation.weather

# Get temperature directly in both Celsius and Fahrenheit
temp_celsius = weather.temperature('celsius')['temp']
temp_fahrenheit = weather.temperature('fahrenheit')['temp']

print(f"Temperature in Celsius: {temp_celsius}°C, Fahrenheit: {temp_fahrenheit}°F")


Temperature in Celsius: 13.17°C, Fahrenheit: 55.71°F
